In [1]:
import csv
import pandas as pd
import numpy as np
import re
import datetime as dt
import time

In [2]:
df = pd.read_csv("Seattle_Police_Department_911_Incident_Response.csv", parse_dates=[0], infer_datetime_format=True)

In [3]:
# df['Event Clearance Date']

In [4]:
df['fullDate'] = df['Event Clearance Date'].apply(lambda x: str(x).split(' ')[0])
# df['eventClearanceSeconds'] = pd.DatetimeIndex(df['Event Clearance Date']).hour *3600 + pd.DatetimeIndex(df['Event Clearance Date']).minute *60
df

,CAD CDW ID,CAD Event Number,General Offense Number,Event Clearance Code,Event Clearance Description,Event Clearance SubGroup,Event Clearance Group,Event Clearance Date,Hundred Block Location,District/Sector,Zone/Beat,Census Tract,Longitude,Latitude,Incident Location,Initial Type Description,Initial Type Subgroup,Initial Type Group,At Scene Time,fullDate
0,﻿15736,10000246357,2010246357,242.0,FIGHT DISTURBANCE,DISTURBANCES,DISTURBANCES,07/17/2010 08:49:00 PM,3XX BLOCK OF PINE ST,M,M2,8100.2001,-122.338147,47.610975,"(47.610975163, -122.338146748)",NaN,NaN,NaN,NaN,07/17/2010
1,15737,10000246471,2010246471,65.0,THEFT - MISCELLANEOUS,THEFT,OTHER PROPERTY,07/17/2010 08:50:00 PM,36XX BLOCK OF DISCOVERY PARK BLVD,Q,Q1,5700.1012,-122.404613,47.658325,"(47.658324899, -122.404612874)",NaN,NaN,NaN,NaN,07/17/2010
2,15738,10000246255,2010246255,250.0,"MISCHIEF, NUISANCE COMPLAINTS","NUISANCE, MISCHIEF COMPLAINTS","NUISANCE, MISCHIEF",07/17/2010 08:55:00 PM,21XX BLOCK OF 3RD AVE,M,M2,7200.2025,-122.342843,47.613551,"(47.613551471, -122.342843234)",NaN,NaN,NaN,NaN,07/17/2010
3,15739,10000246473,2010246473,460.0,TRAFFIC (MOVING) VIOLATION,TRAFFIC RELATED CALLS,TRAFFIC RELATED CALLS,07/17/2010 09:00:00 PM,7XX BLOCK OF ROY ST,D,D1,7200.1002,-122.341847,47.625401,"(47.625401388, -122.341846999)",NaN,NaN,NaN,NaN,07/17/2010
4,15740,10000246330,2010246330,250.0,"MISCHIEF, NUISANCE COMPLAINTS","NUISANCE, MISCHIEF COMPLAINTS","NUISANCE, MISCHIEF",07/17/2010 09:00:00 PM,9XX BLOCK OF ALOHA ST,D,D1,6700.1009,-122.339709,47.627425,"(47.627424837, -122.339708605)",NaN,NaN,NaN,NaN,07/17/2010
5,15741,10000246477,2010246477,281.0,SUSPICIOUS VEHICLE,SUSPICIOUS CIRCUMSTANCES,SUSPICIOUS CIRCUMSTANCES,07/17/2010 09:02:00 PM,30XX BLOCK OF W GOVERNMENT WAY,Q,Q1,5700.2005,-122.396627,47.661312,"(47.66131158, -122.39662681)",NaN,NaN,NaN,NaN,07/17/2010
6,15742,10000246436,2010246436,220.0,MENTAL COMPLAINT,MENTAL CALL,MENTAL HEALTH,07/17/2010 09:04:00 PM,50XX BLOCK OF 57TH AVE S,R,R3,10200.1012,-122.263332,47.555956,"(47.555955852, -122.263332426)",NaN,NaN,NaN,NaN,07/17/2010
7,15743,10000246472,2010246472,460.0,TRAFFIC (MOVING) VIOLATION,TRAFFIC RELATED CALLS,TRAFFIC RELATED CALLS,07/17/2010 09:05:00 PM,56XX BLOCK OF BROOKLYN AVE NE,U,U3,4400.5003,-122.314166,47.670345,"(47.670345345, -122.3141656)",NaN,NaN,NaN,NaN,07/17/2010
8,15744,10000246493,2010246493,177.0,LIQUOR VIOLATION - INTOXICATED PERSON,LIQUOR VIOLATIONS,LIQUOR VIOLATIONS,07/17/2010 09:09:00 PM,15XX BLOCK OF 4TH AVE,K,K1,8100.2005,-122.336891,47.610440,"(47.610440245, -122.336890515)",NaN,NaN,NaN,NaN,07/17/2010
9,15745,10000246478,2010246478,220.0,MENTAL COMPLAINT,MENTAL CALL,MENTAL HEALTH,07/17/2010 09:17:00 PM,100XX BLOCK OF COLLEGE WAY N,N,N3,1300.1009,-122.334524,47.702332,"(47.702331629, -122.33452433)",NaN,NaN,NaN,NaN,07/17/2010


In [5]:
weather = pd.read_csv("WEATHER.csv")

In [6]:
df.shape

(1333837, 20)

In [7]:
weather

,﻿year,month,date,tempHigh,tempAvg,tempLow,dewHigh,dewAvg,dewLow,humidityHigh,...,seaLvlAvg,seaLvlLow,visibilityHigh,visibilityAvg,visibilityLow,windHigh,windAvg,windHigh.1,precip,events
0,2015,Jan,1,43,35,27,28,25,21,81,...,30.31,30.22,10,10,6,4,0,-,0,
1,2015,Jan,2,44,38,32,39,32,25,86,...,30.16,30.10,10,10,7,12,4,-,0.03,
2,2015,Jan,3,43,38,33,38,35,30,93,...,30.26,30.22,10,7,3,7,2,-,0,Rain
3,2015,Jan,4,54,48,41,48,41,37,89,...,30.14,29.98,10,8,3,21,7,32,0.22,Rain
4,2015,Jan,5,57,56,54,52,50,48,88,...,30.18,30.00,10,8,2,20,14,31,0.07,Rain
5,2015,Jan,6,55,50,46,48,46,43,93,...,30.29,30.26,10,7,0,6,2,-,0.01,Fog
6,2015,Jan,7,50,47,44,46,43,41,93,...,30.28,30.25,10,5,1,6,2,-,0,
7,2015,Jan,8,48,44,39,41,38,36,89,...,30.17,30.12,10,6,0,8,3,-,0,Fog
8,2015,Jan,9,50,44,39,41,39,37,93,...,30.09,30.02,10,4,0,6,2,-,0.01,"Fog , Rain"
9,2015,Jan,10,48,46,44,45,44,41,93,...,30.03,30.00,10,3,1,7,1,-,0.18,Rain


In [8]:
import calendar
weather.shape
list(weather.columns.values)

['\ufeffyear',
 'month',
 'date',
 'tempHigh',
 'tempAvg',
 'tempLow',
 'dewHigh',
 'dewAvg',
 'dewLow',
 'humidityHigh',
 'humidityAvg',
 'humidityLow',
 'seaLvlHigh',
 'seaLvlAvg',
 'seaLvlLow',
 'visibilityHigh',
 'visibilityAvg',
 'visibilityLow',
 'windHigh',
 'windAvg',
 'windHigh.1',
 'precip',
 'events']

In [9]:
months_map = {'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr':'04', 'May': '05', 'Jun': '06', 'Jul': '07', 'Aug': '08', 'Sep': '09', 'Oct':'10', 'Nov': '11', 'Dec': '12'}

In [10]:
weather['m'] = weather['month'].apply(lambda x: months_map[x])

In [11]:
weather[200:]

,﻿year,month,date,tempHigh,tempAvg,tempLow,dewHigh,dewAvg,dewLow,humidityHigh,...,seaLvlLow,visibilityHigh,visibilityAvg,visibilityLow,windHigh,windAvg,windHigh.1,precip,events,m
200,2015,Jul,20,82,74,66,58,54,53,75,...,29.97,10,10,10,13,8,21,0,,07
201,2015,Jul,21,75,68,62,56,52,49,78,...,29.90,10,10,4,9,3,-,0.13,Rain,07
202,2015,Jul,22,77,68,61,52,50,48,70,...,29.86,10,10,10,10,4,-,0,,07
203,2015,Jul,23,81,71,62,51,50,48,65,...,29.91,10,10,10,7,2,-,0,,07
204,2015,Jul,24,77,67,59,59,53,50,76,...,30.02,10,10,7,13,6,24,0.01,Rain,07
205,2015,Jul,25,70,66,62,61,54,49,88,...,30.02,10,10,5,12,4,16,0.02,Rain,07
206,2015,Jul,26,73,66,60,56,52,49,80,...,30.04,10,10,4,8,2,-,0.1,Rain,07
207,2015,Jul,27,77,65,55,54,51,45,84,...,30.11,10,10,6,7,2,-,0.01,,07
208,2015,Jul,28,82,72,61,53,51,47,72,...,30.17,10,10,10,9,3,-,0,,07
209,2015,Jul,29,89,75,62,55,51,48,75,...,30.07,10,10,9,10,3,18,0,,07


In [12]:
weather['day'] = weather['date'].apply(lambda x: ('0' + str(x)) if x < 10 else str(x) )
weather['fullDate'] = weather.apply(lambda x: str(x['m']) + '/' + str(x['day']) + '/' + str(x['\ufeffyear']), axis=1 )
#If the apply doesn't work you may have to pass 'axis=1' after the lambda
#(pass in df['weather'] or whatever your column is called)
weather['clear'] = weather['events'].apply(lambda x: 1 if ('Rain' in str(x) or 'Snow' in str(x) or 'Fog' in str(x) or 'Thunderstorm' in str(x)) else 0)
weather['notClear'] = weather['clear'].apply(lambda x: 1 if x is 0 else 0)

In [13]:
weather[2200:]

,﻿year,month,date,tempHigh,tempAvg,tempLow,dewHigh,dewAvg,dewLow,humidityHigh,...,windHigh,windAvg,windHigh.1,precip,events,m,day,fullDate,clear,notClear
2200,2014,Jan,10,55,51,47,46,42,41,86,...,30,16,41,0.17,Rain,01,10,01/10/2014,1,0
2201,2014,Jan,11,58,52,45,48,42,37,86,...,41,20,56,0.84,Rain,01,11,01/11/2014,1,0
2202,2014,Jan,12,52,47,42,46,41,35,86,...,30,18,39,0.06,Rain,01,12,01/12/2014,1,0
2203,2014,Jan,13,51,51,50,46,45,43,86,...,33,16,45,0,NaN,01,13,01/13/2014,0,1
2204,2014,Jan,14,52,49,45,46,45,42,93,...,13,3,16,T,"Fog , Rain",01,14,01/14/2014,1,0
2205,2014,Jan,15,52,47,42,44,41,39,93,...,13,6,15,0,Fog,01,15,01/15/2014,1,0
2206,2014,Jan,16,44,42,40,41,39,37,96,...,14,6,15,0,Fog,01,16,01/16/2014,1,0
2207,2014,Jan,17,42,40,37,38,37,35,93,...,10,5,13,0,Fog,01,17,01/17/2014,1,0
2208,2014,Jan,18,49,41,33,39,35,30,96,...,14,5,15,0,Fog,01,18,01/18/2014,1,0
2209,2014,Jan,19,43,41,38,40,37,35,93,...,20,6,31,0,Fog,01,19,01/19/2014,1,0


In [14]:
list(df.columns.values)

['CAD CDW ID',
 'CAD Event Number',
 'General Offense Number',
 'Event Clearance Code',
 'Event Clearance Description',
 'Event Clearance SubGroup',
 'Event Clearance Group',
 'Event Clearance Date',
 'Hundred Block Location',
 'District/Sector',
 'Zone/Beat',
 'Census Tract',
 'Longitude',
 'Latitude',
 'Incident Location',
 'Initial Type Description',
 'Initial Type Subgroup',
 'Initial Type Group',
 'At Scene Time',
 'fullDate']

In [15]:
weather.events.unique()

array([' ', 'Rain', 'Fog', 'Fog , Rain', 'Rain , Thunderstorm', 'Snow',
       'Rain , Snow', 'Fog , Rain , Snow', nan, 'Fog , Snow',
       'Fog , Rain , Thunderstorm', 'Rain , Snow , Thunderstorm',
       'Thunderstorm'], dtype=object)

In [20]:
merged = pd.merge(df, weather, on = 'fullDate', how = 'left')

In [17]:
merged[1000000:] #splines

,CAD CDW ID,CAD Event Number,General Offense Number,Event Clearance Code,Event Clearance Description,Event Clearance SubGroup,Event Clearance Group,Event Clearance Date,Hundred Block Location,District/Sector,...,visibilityLow,windHigh,windAvg,windHigh.1,precip,events,m,day,clear,notClear
1000000,1953218,14000064651,201464651,177.0,LIQUOR VIOLATION - INTOXICATED PERSON,LIQUOR VIOLATIONS,LIQUOR VIOLATIONS,03/01/2014 03:43:00 PM,17XX BLOCK OF 11 AV,C,...,10.0,17.0,11.0,21,0.02,"Rain , Snow",03,01,1.0,0.0
1000001,1953221,14000064572,201464572,63.0,THEFT - CAR PROWL,CAR PROWL,CAR PROWL,03/01/2014 04:01:00 PM,SW EDMUNDS ST / 47 AV SW,W,...,10.0,17.0,11.0,21,0.02,"Rain , Snow",03,01,1.0,0.0
1000002,1953222,14000064565,201464565,139.0,GANG GRAFFITI,PROPERTY DAMAGE,PROPERTY DAMAGE,03/01/2014 03:46:00 PM,32XX BLOCK OF NE 45 ST,L,...,10.0,17.0,11.0,21,0.02,"Rain , Snow",03,01,1.0,0.0
1000003,1953223,14000064557,201464557,220.0,MENTAL COMPLAINT,MENTAL CALL,MENTAL HEALTH,03/01/2014 03:22:00 PM,1 AV / YESLER WY,K,...,10.0,17.0,11.0,21,0.02,"Rain , Snow",03,01,1.0,0.0
1000004,1953224,14000064527,201464527,192.0,MISDEMEANOR WARRANT SERVICE,WARRANT CALLS,ARREST,03/01/2014 03:22:00 PM,28XX BLOCK OF SW BARTON ST,F,...,10.0,17.0,11.0,21,0.02,"Rain , Snow",03,01,1.0,0.0
1000005,1953225,14000064503,201464503,201.0,ALARMS - RESIDENTIAL BURGLARY (FALSE),BURGLARY ALARMS (FALSE),FALSE ALARMS,03/01/2014 03:29:00 PM,42XX BLOCK OF E MADISON ST,C,...,10.0,17.0,11.0,21,0.02,"Rain , Snow",03,01,1.0,0.0
1000006,1953227,14000060545,201460545,470.0,PARKING VIOLATION (EXCEPT ABANDONED VEHICLES),PARKING VIOLATIONS,TRAFFIC RELATED CALLS,02/25/2014 09:15:00 PM,53XX BLOCK OF 15 AV S,R,...,0.0,16.0,10.0,18,0.01,NaN,02,25,0.0,1.0
1000007,1953229,14000064716,201464716,201.0,ALARMS - RESIDENTIAL BURGLARY (FALSE),BURGLARY ALARMS (FALSE),FALSE ALARMS,03/01/2014 04:16:00 PM,79XX BLOCK OF 13 AV SW,F,...,10.0,17.0,11.0,21,0.02,"Rain , Snow",03,01,1.0,0.0
1000008,1953230,14000064713,201464713,460.0,TRAFFIC (MOVING) VIOLATION,TRAFFIC RELATED CALLS,TRAFFIC RELATED CALLS,03/01/2014 04:12:00 PM,4 AV / JEFFERSON ST,K,...,10.0,17.0,11.0,21,0.02,"Rain , Snow",03,01,1.0,0.0
1000009,1953231,14000064709,201464709,450.0,DRIVING WHILE UNDER INFLUENCE (DUI),TRAFFIC RELATED CALLS,TRAFFIC RELATED CALLS,03/01/2014 04:06:00 PM,15 AV NE / NE 145 ST,L,...,10.0,17.0,11.0,21,0.02,"Rain , Snow",03,01,1.0,0.0


In [18]:
merged['Event Clearance Group'].unique()

array(['DISTURBANCES', 'OTHER PROPERTY', 'NUISANCE, MISCHIEF ',
       'TRAFFIC RELATED CALLS', 'SUSPICIOUS CIRCUMSTANCES',
       'MENTAL HEALTH', 'LIQUOR VIOLATIONS', 'TRESPASS', 'ASSAULTS',
       'NARCOTICS COMPLAINTS', 'ACCIDENT INVESTIGATION', 'SHOPLIFTING',
       'PROWLER', 'BURGLARY', 'HAZARDS', 'PROPERTY DAMAGE', 'CAR PROWL',
       'BIKE', 'PROSTITUTION', 'MISCELLANEOUS MISDEMEANORS',
       'ANIMAL COMPLAINTS', 'THREATS, HARASSMENT', 'PERSON DOWN/INJURY',
       'AUTO THEFTS', 'PERSONS - LOST, FOUND, MISSING', 'FALSE ALARMS',
       'ROBBERY', 'ARREST', 'FALSE ALACAD', 'PROPERTY - MISSING, FOUND',
       'RECKLESS BURNING', 'BEHAVIORAL HEALTH', 'WEAPONS CALLS',
       'NUISANCE, MISCHIEF', 'LEWD CONDUCT', 'FRAUD CALLS', 'HARBOR CALLS',
       'MOTOR VEHICLE COLLISION INVESTIGATION', 'DRIVE BY (NO INJURY)',
       'OTHER VICE', 'FAILURE TO REGISTER (SEX OFFENDER)', 'HOMICIDE',
       'VICE CALLS', 'PUBLIC GATHERINGS', nan], dtype=object)

In [19]:
list(merged.columns.values)

['CAD CDW ID',
 'CAD Event Number',
 'General Offense Number',
 'Event Clearance Code',
 'Event Clearance Description',
 'Event Clearance SubGroup',
 'Event Clearance Group',
 'Event Clearance Date',
 'Hundred Block Location',
 'District/Sector',
 'Zone/Beat',
 'Census Tract',
 'Longitude',
 'Latitude',
 'Incident Location',
 'Initial Type Description',
 'Initial Type Subgroup',
 'Initial Type Group',
 'At Scene Time',
 'fullDate',
 '\ufeffyear',
 'month',
 'date',
 'tempHigh',
 'tempAvg',
 'tempLow',
 'dewHigh',
 'dewAvg',
 'dewLow',
 'humidityHigh',
 'humidityAvg',
 'humidityLow',
 'seaLvlHigh',
 'seaLvlAvg',
 'seaLvlLow',
 'visibilityHigh',
 'visibilityAvg',
 'visibilityLow',
 'windHigh',
 'windAvg',
 'windHigh.1',
 'precip',
 'events',
 'm',
 'day',
 'clear',
 'notClear']

In [21]:
merged.max(0)

CAD Event Number          1.600043e+10
General Offense Number    2.012221e+10
Event Clearance Code      6.170000e+02
Census Tract              2.650010e+04
Longitude                -1.222174e+02
Latitude                  4.777798e+01
eventClearanceSeconds     8.634000e+04
﻿year                     2.016000e+03
date                      3.100000e+01
tempHigh                  9.800000e+01
tempAvg                   8.300000e+01
tempLow                   7.000000e+01
dewHigh                   6.600000e+01
dewAvg                    6.300000e+01
dewLow                    6.100000e+01
humidityHigh              1.000000e+02
humidityAvg               9.700000e+01
humidityLow               9.300000e+01
seaLvlHigh                3.087000e+01
seaLvlAvg                 3.082000e+01
seaLvlLow                 3.076000e+01
visibilityHigh            1.000000e+01
visibilityAvg             1.000000e+01
visibilityLow             1.000000e+01
windHigh                  7.600000e+01
windAvg                  

In [18]:
df['eventClearanceSeconds'] = pd.DatetimeIndex(df['Event Clearance Date']).hour *3600 + pd.DatetimeIndex(df['Event Clearance Date']).minute *60


In [19]:
df

,CAD CDW ID,CAD Event Number,General Offense Number,Event Clearance Code,Event Clearance Description,Event Clearance SubGroup,Event Clearance Group,Event Clearance Date,Hundred Block Location,District/Sector,...,Census Tract,Longitude,Latitude,Incident Location,Initial Type Description,Initial Type Subgroup,Initial Type Group,At Scene Time,fullDate,eventClearanceSeconds
0,﻿15736,10000246357,2010246357,242.0,FIGHT DISTURBANCE,DISTURBANCES,DISTURBANCES,07/17/2010 08:49:00 PM,3XX BLOCK OF PINE ST,M,...,8100.2001,-122.338147,47.610975,"(47.610975163, -122.338146748)",NaN,NaN,NaN,NaN,07/17/2010,74940.0
1,15737,10000246471,2010246471,65.0,THEFT - MISCELLANEOUS,THEFT,OTHER PROPERTY,07/17/2010 08:50:00 PM,36XX BLOCK OF DISCOVERY PARK BLVD,Q,...,5700.1012,-122.404613,47.658325,"(47.658324899, -122.404612874)",NaN,NaN,NaN,NaN,07/17/2010,75000.0
2,15738,10000246255,2010246255,250.0,"MISCHIEF, NUISANCE COMPLAINTS","NUISANCE, MISCHIEF COMPLAINTS","NUISANCE, MISCHIEF",07/17/2010 08:55:00 PM,21XX BLOCK OF 3RD AVE,M,...,7200.2025,-122.342843,47.613551,"(47.613551471, -122.342843234)",NaN,NaN,NaN,NaN,07/17/2010,75300.0
3,15739,10000246473,2010246473,460.0,TRAFFIC (MOVING) VIOLATION,TRAFFIC RELATED CALLS,TRAFFIC RELATED CALLS,07/17/2010 09:00:00 PM,7XX BLOCK OF ROY ST,D,...,7200.1002,-122.341847,47.625401,"(47.625401388, -122.341846999)",NaN,NaN,NaN,NaN,07/17/2010,75600.0
4,15740,10000246330,2010246330,250.0,"MISCHIEF, NUISANCE COMPLAINTS","NUISANCE, MISCHIEF COMPLAINTS","NUISANCE, MISCHIEF",07/17/2010 09:00:00 PM,9XX BLOCK OF ALOHA ST,D,...,6700.1009,-122.339709,47.627425,"(47.627424837, -122.339708605)",NaN,NaN,NaN,NaN,07/17/2010,75600.0
5,15741,10000246477,2010246477,281.0,SUSPICIOUS VEHICLE,SUSPICIOUS CIRCUMSTANCES,SUSPICIOUS CIRCUMSTANCES,07/17/2010 09:02:00 PM,30XX BLOCK OF W GOVERNMENT WAY,Q,...,5700.2005,-122.396627,47.661312,"(47.66131158, -122.39662681)",NaN,NaN,NaN,NaN,07/17/2010,75720.0
6,15742,10000246436,2010246436,220.0,MENTAL COMPLAINT,MENTAL CALL,MENTAL HEALTH,07/17/2010 09:04:00 PM,50XX BLOCK OF 57TH AVE S,R,...,10200.1012,-122.263332,47.555956,"(47.555955852, -122.263332426)",NaN,NaN,NaN,NaN,07/17/2010,75840.0
7,15743,10000246472,2010246472,460.0,TRAFFIC (MOVING) VIOLATION,TRAFFIC RELATED CALLS,TRAFFIC RELATED CALLS,07/17/2010 09:05:00 PM,56XX BLOCK OF BROOKLYN AVE NE,U,...,4400.5003,-122.314166,47.670345,"(47.670345345, -122.3141656)",NaN,NaN,NaN,NaN,07/17/2010,75900.0
8,15744,10000246493,2010246493,177.0,LIQUOR VIOLATION - INTOXICATED PERSON,LIQUOR VIOLATIONS,LIQUOR VIOLATIONS,07/17/2010 09:09:00 PM,15XX BLOCK OF 4TH AVE,K,...,8100.2005,-122.336891,47.610440,"(47.610440245, -122.336890515)",NaN,NaN,NaN,NaN,07/17/2010,76140.0
9,15745,10000246478,2010246478,220.0,MENTAL COMPLAINT,MENTAL CALL,MENTAL HEALTH,07/17/2010 09:17:00 PM,100XX BLOCK OF COLLEGE WAY N,N,...,1300.1009,-122.334524,47.702332,"(47.702331629, -122.33452433)",NaN,NaN,NaN,NaN,07/17/2010,76620.0


In [34]:
sample = merged.sample(n=50,random_state = 13)
sample.shape

(50, 48)

In [35]:
sample['eventClearanceSeconds']

214484     75420.0
686701     62640.0
56409      79560.0
505841     45360.0
1285608    85020.0
754422     50700.0
741225     39060.0
292533     53940.0
664786     45480.0
1320962        NaN
230890     70740.0
1166156    54000.0
383349     40620.0
1259016    39300.0
72253      85440.0
858691     55260.0
979181     65280.0
1091758    33180.0
971912     79500.0
135596     25320.0
797288     81840.0
1280098    75600.0
1195515     1740.0
979824     65280.0
1265911    12540.0
607914      9000.0
228043     67860.0
518108     17220.0
420249     78600.0
367822     33360.0
1251679    28800.0
219309     39120.0
757241     71700.0
1162611    67560.0
1219724    70080.0
1108717    81480.0
745748      8460.0
903256     81180.0
761114     29880.0
87677      21480.0
371669      2400.0
420087     62460.0
707031     34140.0
679378     10620.0
1114086    64020.0
668921      2040.0
171225     64800.0
539914     14220.0
455820     49980.0
868567     46680.0
Name: eventClearanceSeconds, dtype: float64

In [45]:
from sklearn import linear_model
sample = sample[np.isfinite(df['eventClearanceSeconds'])] 
sample['secondsInDay'] = sample['eventClearanceSeconds'].apply(lambda x: int(x))
features = ['Event Clearance Group', 'secondsInDay']
X = sample[features]
X

C:\Users\Nhungh\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  from ipykernel import kernelapp as app


,Event Clearance Group,secondsInDay
214484,TRAFFIC RELATED CALLS,75420
686701,LIQUOR VIOLATIONS,62640
56409,LIQUOR VIOLATIONS,79560
505841,TRAFFIC RELATED CALLS,45360
1285608,DISTURBANCES,85020
754422,TRAFFIC RELATED CALLS,50700
741225,FRAUD CALLS,39060
292533,DISTURBANCES,53940
664786,MISCELLANEOUS MISDEMEANORS,45480
230890,TRAFFIC RELATED CALLS,70740


In [46]:

y = df['Event Clearance Group']
lr = linear_model.LogisticRegression()
lr.fit(X, y)
preds = lr.predict(X)

ValueError: could not convert string to float: 'FALSE ALACAD'